(train_transformers_accelerate_example)=

# Fine-tune a 🤗 Transformers model

This notebook is based on [an official 🤗 notebook - "How to fine-tune a model on text classification"](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb). The main aim of this notebook is to show the process of conversion from vanilla 🤗 to Ray Train without changing the training logic unless necessary.

In this notebook, we will:
1. [Set up Ray](#setup)
2. [Load the dataset](#load)
3. [Preprocess the dataset with Ray Data](#preprocess)
4. [Run the training with Ray Train](#train)
5. [Optionally, share the model with the community](#share)

Uncomment and run the following line in order to install all the necessary dependencies (this notebook is being tested with `transformers==4.19.1`):

In [1]:
#! pip install "datasets" "transformers>=4.19.0" "torch>=1.10.0" "mlflow"

## Set up Ray <a name="setup"></a>

We will use `ray.init()` to initialize a local cluster. By default, this cluster will be comprised of only the machine you are running this notebook on. You can also run this notebook on an Anyscale cluster.

In [ ]:
from pprint import pprint
import ray

ray.init()

We can check the resources our cluster is composed of. If you are running this notebook on your local machine or Google Colab, you should see the number of CPU cores and GPUs available on the said machine.

In [3]:
pprint(ray.cluster_resources())

{'CPU': 64.0,
 'GPU': 4.0,
 'accelerator_type:None': 4.0,
 'memory': 274877906944.0,
 'node:10.0.19.101': 1.0,
 'node:10.0.24.75': 1.0,
 'node:10.0.42.45': 1.0,
 'node:10.0.9.214': 1.0,
 'node:__internal_head__': 1.0,
 'object_store_memory': 77016771377.0}


In this notebook, we will see how to fine-tune a [🤗 Transformers](https://github.com/huggingface/transformers) model for one of the text classification task of the [GLUE Benchmark](https://gluebenchmark.com/). We will be running the training using Ray Train.

You can change those two variables to control whether the training (which we will get to later) uses CPUs or GPUs, and how many workers should be spawned. Each worker will claim one CPU or GPU. Make sure not to request more resources than the resources present!

By default, we will run the training with one GPU worker.

In [4]:
use_gpu = True  # set this to False to run on CPUs
num_workers = 1  # set this to number of GPUs/CPUs you want to use

## Fine-tuning a model on a text classification task

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences. If you would like to learn more, refer to the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb).

Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [5]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [6]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

### Loading the dataset <a name="load"></a>

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions.

As Ray AIR doesn't provide integrations for 🤗 Datasets yet, we will simply run the normal 🤗 Datasets code to load the dataset from the Hub.

In [7]:
from datasets import load_dataset

actual_task = "mnli" if task == "mnli-mm" else task
datasets = load_dataset("glue", actual_task)

Reusing dataset glue (/home/ray/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation, and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

We will also need the metric. In order to avoid serialization errors, we will load the metric inside the training workers later. Therefore, now we will just define the function we will use.

In [8]:
from datasets import load_metric

def load_metric_fn():
    return load_metric('glue', actual_task)

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric).

### Preprocessing the data with Ray Data <a name="preprocess"></a>

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers' `Tokenizer`, which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure that:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [10]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

Instead of using 🤗 Dataset objects directly, we will convert them to [Ray Data](data). Both are backed by Arrow tables, so the conversion is straightforward. We will use the built-in {meth}`~ray.data.from_huggingface` function.

In [11]:
import ray.data

ray_datasets = {
    "train": ray.data.from_huggingface(datasets["train"]),
    "validation": ray.data.from_huggingface(datasets["validation"]),
    "test": ray.data.from_huggingface(datasets["test"]),
}
ray_datasets

{'train': MaterializedDataset(
    num_blocks=1,
    num_rows=8551,
    schema={sentence: string, label: int64, idx: int32}
 ),
 'validation': MaterializedDataset(
    num_blocks=1,
    num_rows=1043,
    schema={sentence: string, label: int64, idx: int32}
 ),
 'test': MaterializedDataset(
    num_blocks=1,
    num_rows=1063,
    schema={sentence: string, label: int64, idx: int32}
 )}

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

We use `map_batches` which will map the function to the datasets in a distributed fashion.

In [12]:
import numpy as np
from typing import Dict


def preprocess_function(examples: Dict[str, np.array]):
    # if we only have one column, we are inferring.
    # no need to tokenize in that case. 
    if len(examples) == 1:
        return examples
    
    return_dict = {}
    return_dict["labels"] = examples["label"]

    sentence1_key, sentence2_key = task_to_keys[task]
    if sentence2_key is None:
        ret = tokenizer(list(examples[sentence1_key]), truncation=True, padding="max_length", return_tensors="np")
    else:
        ret = tokenizer(list(examples[sentence1_key]), list(examples[sentence2_key]), truncation=True, padding="max_length", return_tensors="np")
    return_dict.update(ret)
    return return_dict

for split, dataset in ray_datasets.items():
    ray_datasets[split] = dataset.map_batches(preprocess_function, batch_format="numpy")

In [13]:
for batch in ray_datasets["train"].iter_batches(batch_size=3):
    print(batch)
    break

2023-09-05 16:09:50,547	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(preprocess_function)]
2023-09-05 16:09:50,548	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-09-05 16:09:50,549	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

{'labels': array([1, 1, 1]), 'input_ids': array([[ 101, 2256, 2814, ...,    0,    0,    0],
       [ 101, 2028, 2062, ...,    0,    0,    0],
       [ 101, 2028, 2062, ...,    0,    0,    0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}


### Fine-tuning the model with Ray Train <a name="train"></a>

Now that our data is ready, we can download the pretrained model and fine-tune it.

Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class.

We will not go into details about each specific component of the training (see the [original notebook](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb) for that). The tokenizer is the same as we have used to encoded the dataset before.

The main difference when using the Ray Train is that we need to define our our training logics a function (`train_func`). That function will be passed to the {class}`~ray.train.torch.TorchTrainer` and will run on every Ray worker. The training will then proceed by the means of PyTorch DDP.

Make sure that you initialize the model, metric, and tokenizer inside that function. Otherwise, you may run into serialization errors.

Furthermore, `push_to_hub=True` is not yet supported. Ray will, however, checkpoint the model at every epoch, allowing you to push it to hub manually. We will do that after the training.

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch
import ray.train

from ray.train.huggingface.transformers import prepare_trainer, RayTrainReportCallback

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
name = f"{model_name}-finetuned-{task}"

# Calculate the maximum steps per epoch based on the number of rows in the training dataset.
# Make sure to scale by the total number of training workers and the per device batch size.
max_steps_per_epoch = ray_datasets["train"].count() // batch_size

def train_func(config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")
    metric = load_metric_fn()
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

    train_ds = ray.train.get_dataset_shard("train")
    eval_ds = ray.train.get_dataset_shard("eval")
    
    batch_size_per_worker = batch_size // num_workers

    train_ds_iterable = train_ds.iter_torch_batches(batch_size=batch_size_per_worker)
    eval_ds_iterable = eval_ds.iter_torch_batches(batch_size=batch_size_per_worker)

    print("max_steps_per_epoch: ", max_steps_per_epoch)
    
    args = TrainingArguments(
        name,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size=batch_size_per_worker,
        per_device_eval_batch_size=batch_size_per_worker,
        learning_rate=config.get("learning_rate", 2e-5),
        num_train_epochs=config.get("epochs", 2),
        weight_decay=config.get("weight_decay", 0.01),
        push_to_hub=False,
        max_steps=max_steps_per_epoch * config.get("epochs", 2),
        # disable_tqdm=True,  # declutter the output a little
        no_cuda=not use_gpu,  # you need to explicitly set no_cuda if you want CPUs
        report_to="none",
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_ds_iterable,
        eval_dataset=eval_ds_iterable,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.add_callback(RayTrainReportCallback())

    trainer = prepare_trainer(trainer)

    print("Starting training")
    trainer.train()

2023-09-05 16:09:54.649811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 16:09:54.796019: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-05 16:09:55.553285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-05 16:09:55.553361: W tensorflow/

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

With our `train_func` complete, we can now instantiate the {class}`~ray.train.torch.TorchTrainer`. Aside from the function, we set the `scaling_config`, controlling the amount of workers and resources used, and the `datasets` we will use for training and evaluation.

In [15]:
from ray.train.torch import TorchTrainer
from ray.train import RunConfig, ScalingConfig, CheckpointConfig

trainer = TorchTrainer(
    train_func,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    datasets={
        "train": ray_datasets["train"],
        "eval": ray_datasets["validation"],
    },
    run_config=RunConfig(
        storage_path="/mnt/cluster_storage/ray_results",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
)


Finally, we call the `fit` method to start training with Ray AIR. We will save the `Result` object to a variable so we can access metrics and checkpoints.

In [16]:
result = trainer.fit()

(TrainTrainable pid=15827, ip=10.0.24.75) 2023-09-05 16:10:05.373103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
(TrainTrainable pid=15827, ip=10.0.24.75) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(TrainTrainable pid=15827, ip=10.0.24.75) 2023-09-05 16:10:05.542473: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(TrainTrainable pid=15827, ip=10.0.24.75) 2023-09-05 16:10:06.355419: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.s

(RayTrainWorker pid=15887, ip=10.0.24.75) Is CUDA available: True
(RayTrainWorker pid=15887, ip=10.0.24.75) max_steps_per_epoch:  534


(RayTrainWorker pid=15887, ip=10.0.24.75) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
(RayTrainWorker pid=15887, ip=10.0.24.75) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(RayTrainWorker pid=15887, ip=10.0.24.75) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(RayTrainWorker pid=15887, ip=10.0.24.75) Some weights of DistilBertForSequenceClas

(RayTrainWorker pid=15887, ip=10.0.24.75) Starting training


(RayTrainWorker pid=15887, ip=10.0.24.75) max_steps is given, it will override any value given in num_train_epochs
(RayTrainWorker pid=15887, ip=10.0.24.75) /home/ray/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(RayTrainWorker pid=15887, ip=10.0.24.75)   warnings.warn(
(RayTrainWorker pid=15887, ip=10.0.24.75) ***** Running training *****
(RayTrainWorker pid=15887, ip=10.0.24.75)   Num examples = 17088
(RayTrainWorker pid=15887, ip=10.0.24.75)   Num Epochs = 9223372036854775807
(RayTrainWorker pid=15887, ip=10.0.24.75)   Instantaneous batch size per device = 16
(RayTrainWorker pid=15887, ip=10.0.24.75)   Total train batch size (w. parallel, distributed & accumulation) = 16
(RayTrainWorker pid=15887, ip=10.0.24.75)   Gradient Accumulation steps 

...

(pid=15938, ip=10.0.24.75) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(RayTrainWorker pid=15887, ip=10.0.24.75) {'eval_loss': 0.5601133108139038, 'eval_matthews_correlation': 0.4373748188096333, 'eval_runtime': 15.9315, 'eval_samples_per_second': 65.468, 'eval_steps_per_second': 4.143, 'epoch': 1.5}


100%|██████████| 1068/1068 [13:32<00:00,  1.50it/s]Saving model checkpoint to distilbert-base-uncased-finetuned-cola/checkpoint-1068
(RayTrainWorker pid=15887, ip=10.0.24.75) Configuration saved in distilbert-base-uncased-finetuned-cola/checkpoint-1068/config.json
(RayTrainWorker pid=15887, ip=10.0.24.75) Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-1068/pytorch_model.bin
(RayTrainWorker pid=15887, ip=10.0.24.75) tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-1068/tokenizer_config.json
(RayTrainWorker pid=15887, ip=10.0.24.75) Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-1068/special_tokens_map.json


(RayTrainWorker pid=15887, ip=10.0.24.75) {'train_runtime': 818.7099, 'train_samples_per_second': 20.872, 'train_steps_per_second': 1.304, 'train_loss': 0.46155189485585646, 'epoch': 1.5}


(RayTrainWorker pid=15887, ip=10.0.24.75) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/mnt/cluster_storage/ray_results/TorchTrainer_2023-09-05_16-10-01/TorchTrainer_56f64_00000_0_2023-09-05_16-10-01/checkpoint_000001)
(RayTrainWorker pid=15887, ip=10.0.24.75) 
(RayTrainWorker pid=15887, ip=10.0.24.75) 
(RayTrainWorker pid=15887, ip=10.0.24.75) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=15887, ip=10.0.24.75) 
(RayTrainWorker pid=15887, ip=10.0.24.75) 
100%|██████████| 1068/1068 [13:38<00:00,  1.30it/s] 
2023-09-05 16:24:00,646	WARNING experiment_state.py:371 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or 

You can use the returned `Result` object to access metrics and the Ray Train `Checkpoint` associated with the last iteration.

In [17]:
result

Result(
  metrics={'loss': 0.3826, 'learning_rate': 0.0, 'epoch': 1.5, 'step': 1068, 'eval_loss': 0.5601133108139038, 'eval_matthews_correlation': 0.4373748188096333, 'eval_runtime': 15.9315, 'eval_samples_per_second': 65.468, 'eval_steps_per_second': 4.143},
  path='/mnt/cluster_storage/ray_results/TorchTrainer_2023-09-05_16-10-01/TorchTrainer_56f64_00000_0_2023-09-05_16-10-01',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/mnt/cluster_storage/ray_results/TorchTrainer_2023-09-05_16-10-01/TorchTrainer_56f64_00000_0_2023-09-05_16-10-01/checkpoint_000001)
)

### Tune hyperparameters with Ray Tune <a name="predict"></a>

If we would like to tune any hyperparameters of the model, we can do so by simply passing our `TorchTrainer` into a `Tuner` and defining the search space.

We can also take advantage of the advanced search algorithms and schedulers provided by Ray Tune. In this example, we will use an `ASHAScheduler` to aggresively terminate underperforming trials.

In [25]:
from ray import tune
from ray.tune import Tuner
from ray.tune.schedulers.async_hyperband import ASHAScheduler

tune_epochs = 4
tuner = Tuner(
    trainer,
    param_space={
        "train_loop_config": {
            "learning_rate": tune.grid_search([2e-5, 2e-4, 2e-3, 2e-2]),
            "epochs": tune_epochs,
        }
    },
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
        num_samples=1,
        scheduler=ASHAScheduler(
            max_t=tune_epochs,
        )
    ),
    run_config=RunConfig(
        name="tune_transformers",
        storage_path="/mnt/cluster_storage/ray_results",
        checkpoint_config=CheckpointConfig(num_to_keep=1, checkpoint_score_attribute="eval_loss", checkpoint_score_order="min")
    ),
)

2023-09-05 16:35:56,242	INFO tuner_internal.py:508 -- A `RunConfig` was passed to both the `Tuner` and the `TorchTrainer`. The run config passed to the `Tuner` is the one that will be used.


In [26]:
tune_results = tuner.fit()

(TrainTrainable pid=21888, ip=10.0.42.45) 2023-09-05 16:36:07.866977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
(TrainTrainable pid=21888, ip=10.0.42.45) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(TrainTrainable pid=21888, ip=10.0.42.45) 2023-09-05 16:36:08.011580: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(TrainTrainable pid=21888, ip=10.0.42.45) 2023-09-05 16:36:08.735652: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.s

(RayTrainWorker pid=21951, ip=10.0.42.45) Is CUDA available: True


(RayTrainWorker pid=21951, ip=10.0.42.45) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
(RayTrainWorker pid=21951, ip=10.0.42.45) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(RayTrainWorker pid=21951, ip=10.0.42.45) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(RayTrainWorker pid=21951, ip=10.0.42.45) Some weights of DistilBertForSequenceClas

(RayTrainWorker pid=21951, ip=10.0.42.45) max_steps_per_epoch:  534


(RayTrainWorker pid=21951, ip=10.0.42.45) max_steps is given, it will override any value given in num_train_epochs
(RayTrainWorker pid=21951, ip=10.0.42.45) /home/ray/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(RayTrainWorker pid=21951, ip=10.0.42.45)   warnings.warn(


...

(RayTrainWorker pid=55340) Model weights saved in distilbert-base-uncased-finetuned-cola/checkpoint-2136/pytorch_model.bin
(RayTrainWorker pid=55340) tokenizer config file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2136/tokenizer_config.json
(RayTrainWorker pid=55340) Special tokens file saved in distilbert-base-uncased-finetuned-cola/checkpoint-2136/special_tokens_map.json


(autoscaler +52m2s) [workspace snapshot] New snapshot created successfully (size: 407.01 KB).


(RayTrainWorker pid=55340) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/mnt/cluster_storage/ray_results/tune_transformers/TorchTrainer_fac06_00000_0_learning_rate=0.0000_2023-09-05_16-36-04/checkpoint_000003)
(RayTrainWorker pid=55340) 
(RayTrainWorker pid=55340) 
(RayTrainWorker pid=55340) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=55340) 
(RayTrainWorker pid=55340) 
100%|██████████| 2136/2136 [25:27<00:00,  1.40it/s]
2023-09-05 17:01:50,009	WARNING experiment_state.py:371 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXC

(RayTrainWorker pid=55340) {'train_runtime': 1527.6333, 'train_samples_per_second': 22.372, 'train_steps_per_second': 1.398, 'train_loss': 0.34186658252044566, 'epoch': 3.25}


2023-09-05 17:02:04,184	WARNING experiment_state.py:311 -- The previous sync of the experiment directory to the cloud failed with the error: /home/ray/ray_results/tune_transformers/.tmp_experiment_state_9f3ca371-db4b-45ac-ba16-df266ce6163f
Syncing will be retried.
2023-09-05 17:02:17,769	WARNING experiment_state.py:371 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
2023-09-05 17:02:17,774	INFO tune.py:1154 -- Total run time: 1573.45 seconds (1545.66 seconds for the tuning loop).


We can view the results of the tuning run as a dataframe, and obtain the best result.

In [27]:
tune_results.get_dataframe().sort_values("eval_loss")

,loss,learning_rate,epoch,step,eval_loss,eval_matthews_correlation,eval_runtime,eval_samples_per_second,eval_steps_per_second,timestamp,...,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore,checkpoint_dir_name,config/train_loop_config/learning_rate,config/train_loop_config/epochs,logdir
1,0.6217,0.000150,0.25,535,0.618047,0.000000,16.3166,63.922,4.045,1693957372,...,400.380528,21972,ip-10-0-9-214,10.0.9.214,400.380528,1,checkpoint_000000,0.00020,4,fac06_00001
3,1.1650,0.014991,0.25,535,0.618265,0.000000,16.2647,64.127,4.058,1693957375,...,402.832207,22226,ip-10-0-24-75,10.0.24.75,402.832207,1,checkpoint_000000,0.02000,4,fac06_00003
2,0.6488,0.001499,0.25,535,0.618356,0.000000,16.0909,64.819,4.102,1693957362,...,391.176487,21888,ip-10-0-42-45,10.0.42.45,391.176487,1,checkpoint_000000,0.00200,4,fac06_00002
0,0.1941,0.000000,3.25,2136,0.752667,0.539651,16.5097,63.175,3.998,1693958509,...,1537.989638,55234,ip-10-0-19-101,10.0.19.101,1537.989638,4,checkpoint_000003,0.00002,4,fac06_00000


In [28]:
best_result = tune_results.get_best_result()

### Share the model <a name="share"></a>

To be able to share your model with the community, there are a few more steps to follow.

We have conducted the training on the Ray cluster, but share the model from the local enviroment - this will allow us to easily authenticate.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [23]:
# !apt install git-lfs

Now, load the model and tokenizer locally, and recreate the 🤗 Transformers `Trainer`:

In [ ]:
from ray.train.huggingface import LegacyTransformersCheckpoint

checkpoint = LegacyTransformersCheckpoint.from_checkpoint(result.checkpoint)
hf_trainer = checkpoint.get_model(model=AutoModelForSequenceClassification)

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
hf_trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```

## Next steps

- {ref}`End-to-end: Offline Batch Inference <batch_inference_home>`